<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#OLS" data-toc-modified-id="OLS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>OLS</a></span><ul class="toc-item"><li><span><a href="#Unpooled" data-toc-modified-id="Unpooled-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Unpooled</a></span><ul class="toc-item"><li><span><a href="#All-logs" data-toc-modified-id="All-logs-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>All logs</a></span></li><li><span><a href="#Logs-or-squares" data-toc-modified-id="Logs-or-squares-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Logs or squares</a></span></li></ul></li><li><span><a href="#Pooled" data-toc-modified-id="Pooled-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Pooled</a></span></li></ul></li><li><span><a href="#Linear-model-with-elastic-net-regularization" data-toc-modified-id="Linear-model-with-elastic-net-regularization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Linear model with elastic net regularization</a></span><ul class="toc-item"><li><span><a href="#Unpooled" data-toc-modified-id="Unpooled-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Unpooled</a></span></li><li><span><a href="#Pooled" data-toc-modified-id="Pooled-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Pooled</a></span></li></ul></li><li><span><a href="#Save-performance" data-toc-modified-id="Save-performance-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Save performance</a></span></li></ul></div>

In [95]:
# Importing libraries
import pdb 
import glob
import copy
import math
# import pickle
import csv

import numpy as np
import pandas as pd
import scipy as sp

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
 
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, \
    StratifiedShuffleSplit, cross_val_score, KFold
#     GridSearchCV, RandomizedSearchCV

# from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, \
    ElasticNet
#     LogisticRegressionCV, SGDClassifier
# from sklearn.svm import SVC, LinearSVC
# from sklearn.ensemble import RandomForestClassifier, \
#     ExtraTreesClassifier

from sklearn.externals import joblib
from sklearn.utils import resample
# from sklearn.utils.fixes import signature

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval, pyll
# import xgboost as xgb
# from xgboost.sklearn import XGBRegressor

# Set up pandas table display
pd.set_option('display.width', 120)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

# Set plotting options
sns.set() # Use seaborn defaults for plotting
%matplotlib inline 

# Load line profiler
# %load_ext line_profiler

# Adjust number of CPU cores to use
N_JOBS=3

In [96]:
# Load data from previous notebook
all_data = pd.read_csv('../../data_raw/all_data.csv')

## Preprocessing

Let's first take another look at the data to make sure everything is alright.

In [97]:
# Inspect data
all_data.head(3)

,Unnamed: 0,age,allsuites,boutique,casino,cbd,category,conference,convention,floors,golf,indoorcorridors,landsf,largestmeetingspace,location,msa,multiproperty,new,operation,portfolio,quarter,resort,restaurant,rooms,saleaffiliation,saleprice,sizesf,ski,spa,id,tract,year,log1_landsf,log1_age,log1_largestmeetingspace,log_saleprice,log_sizesf,log_rooms,log_floors,year_2,year_3,age_2,age_3,floors_2,land,selfrun
0,0,48.0,0.0,0.0,0.0,0.0,Economy Class,0.0,0.0,2.0,0.0,0.0,130301.0,2000.0,Small Metro/Town,"Dothan, AL",0,0,Independent,0,2,0.0,1.0,102.0,Independent,1000000.0,42446.0,0.0,0.0,10,"Dothan/Enterprise, AL",2009,11.777610,3.891820,7.601402,13.815511,10.655988,4.624973,0.693147,4036081,8108486729,2304.0,110592.0,4.0,1,0
1,1,29.0,0.0,0.0,0.0,0.0,Economy Class,0.0,0.0,3.0,0.0,1.0,361548.0,3000.0,Small Metro/Town,"Florence-Muscle Shoals, AL",0,0,Independent,0,2,0.0,1.0,202.0,Independent,1000000.0,151421.0,0.0,0.0,31,Alabama North Area,2010,12.798153,3.401197,8.006701,13.815511,11.927819,5.308268,1.098612,4040100,8120601000,841.0,24389.0,9.0,1,0
2,2,26.0,0.0,0.0,0.0,0.0,Upscale Class,0.0,0.0,4.0,0.0,1.0,357627.0,5400.0,Suburban,"Phoenix-Mesa-Scottsdale, AZ",0,0,Franchise,0,4,0.0,1.0,248.0,Crowne Plaza,24000000.0,142978.0,0.0,0.0,39,"Black Canyon Corridor, AZ",2007,12.787249,3.295837,8.594339,16.993564,11.870446,5.513429,1.386294,4028049,8084294343,676.0,17576.0,16.0,1,0


This shows that we need to drop the first column, which probably was a (meaningless) index that mistakenly was made a column when importing the data. Also, I will drop the binary variable "resort", since resort is also a category of the variable "location".

In [98]:
all_data = all_data.drop(['Unnamed: 0', 'resort', 'land'], axis=1)

### Splitting the data
Let's now start with the actual preprocessing. As mentioned in the introduction and problem statement, an important goal of this project is to show that practitioners influenced predominantly by applied statistics should adopt common practices from machine learning to validate models. (While these approaches may have originated in statistics, the point is that they have only become common practice in machine learning.)

The first such technique is the idea of withholding data (called a test set) when training the model, and then using these withheld data to assess performance. There are two main approaches of how to divide the data into training and test set: We can either split the data randomly, or we can choose a point in time and use all the data originating from an earlier point in time as the training set, and use the later data as the test set.  The former approach works best for independent data, and the latter works best for time-series data.

Our data fall somewhere in between: They are not fully independent, because observations that are closer in time are likely to be more similar, even if we take into account all available predictors. (This is often described as having correlated errors.)  On the other hand, though, we are not dealing with a true time-series, because we are not following the same units over time. (An example of a pooled time-series would be repeated sales data.)  In this true time-series case, we can commonly observe such an extreme serial correlation of errors that this autoregressive process contains more useful information for prediction than the actual features do.

I decided on a compromise: For the test set, I use the latest approximately 20% of the data, which cover the last 1.5 years of the sample. This should make sure we are not overly optimistic, as this makes our calculation of the test accuracy more similar to making predictions for use in practice, when we will probably not have a good estimate of the current year's average price yet. (When I calculate the predictions, I simply treat them as if they were from the last year from which we had data, which should give a better estimate than leaving out the information for year completely, because presumably this year's price will be more similar to last year's prize than to the average price of the last  25 years.)

A disadvantage of this strategy is that the newest portion of the data may not be representative of all years, for example because it is drawn only from a particular point in the business cycle.  However, this is likely not a severe problem for our case, because the business cycle presumably mainly influences the level of the price (captured, in the case of a linear regression model, by the intercept) while the price determinants should stay pretty stable otherwise. To be on the safe side, I do split the data randomly when creating a validation set from the training set when performing cross-validation.  The reason that I pick a different strategy here is that we are not interested in the absolute level of the validation error here, only the relative performance of the different hyperparameters.  As a result, it's not a problem that random splitting causes an optimistic bias, because this should affect all hyperparameters values equally.

Let's now go ahead and see which cut-off we should choose in order to get a test set of about 20% (around 1500 observations). To do so, we group the data by year and count the number of observations per year.  Then we compute the cumulative sum, going backwards.

In [99]:
#  Train-test split
# =================

# Count number of observations by year
year_counts_cum = all_data.loc[:, ["year", "id"]] \
    .groupby("year") \
    .count() \
    .sort_index(ascending=False) \
    .cumsum() 

# Properly name counts
year_counts_cum = year_counts_cum. \
    rename(columns={'id': 'Cumulative Sum (counting backwards)'})

# Print the last 5 years
year_counts_cum.head()

,Cumulative Sum (counting backwards)
year,
2017,601
2016,1917
2015,2751
2014,2756
2013,3497


This shows that, if the data is relatively evenly distributed and 2016, we should cut in the middle of that year.  Thus, let's put all observations that fall in the third quarter 2016 or later into the training set.

In [100]:
# Index for test set
test_index = (all_data.loc[
    (all_data.year > 2016) ]
#     | ((all_data.year == 2016) & (all_data.quarter >= 3))] \
    .index
)              
# Index for training set 
train_index = all_data.index.difference(test_index)

# Check length
print(f'Number of observations in test set: {len(test_index)}')

# Get test and train set
data_test = all_data.loc[all_data.index.isin(test_index)]
data_train = all_data.loc[all_data.index.isin(train_index)]

Number of observations in test set: 601


This gives us the test set of about 1300 observations.  Let's check the results.

In [101]:
# Check results
# assert data_train.year.max() == 2016
# assert data_test.year.min() == 2016

Since linear and tree-based models require different preprocessing, we will save another copy to use in another notebook to build a linear model for comparison. 

In [102]:
# # Save data to use with linear model
# joblib.dump(data_train, '../data/data_train_lin')
# joblib.dump(data_test, '../data/data_test_lin')

In [103]:
data_train_lin = data_train
data_test_lin = data_test

In [104]:
# Set year for test set to the most recent year
data_test.year = 2016

C:\Users\1bb71\Anaconda3\envs\hotels2\lib\site-packages\pandas\core\generic.py:5209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


## OLS 

### Unpooled
#### All logs 

First, we will perform one-hot encoding for categorical variables. By contrast to the previous notebook, we will also treat the ordinal variable, hotel category, as categorical. This is necessary because linear models would otherwise wrongly treat the variable as measured on an interval-scale, which is unlikely to be appropriate (this would assume that the difference between each succeeding pair of categories is identical in size).

For the models without regularization, we need to drop the first category of each categorical variable in order to avoid collinearity. This is not so easy to achieve with the python data science stack, because we normally would not want to estimate a model without regularization. Pandas.get_dummies() does offer this possibility, but it then does not give us the option to transform the training data using the same encoding. Thus, the best option is to use scikit-learn's one-hot encoder on each categorical column individually. I will store these in a list, and then drop the first column before concatenating these new data frames. 

In [105]:
# Categorical variables
cats = ['msa', 'tract', 'location', 'category',
        'saleaffiliation', 'operation', 'year', 'quarter']
# Instantiate one-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore',
                    sparse=False)
# Create lists to store encodings for each variable
X_train_cats = []
X_test_cats = []
feature_names_cats = []
# Iterate through categorical variables
for cat in cats:
    # Encode variable i for TEST data
    ohe_train_i = ohe.fit_transform(
                data_train.loc[:, [cat]])
    # Encode variable i for TRAIN data
    ohe_test_i = ohe.transform(
                data_test.loc[:, [cat]])
    
    # Discard first column and save
    X_train_cats.append(ohe_train_i[:,1:])
    X_test_cats.append(ohe_test_i[:,1:])
    
    # Save feature names (w/o first column)
    feature_names_cats.extend(
        ohe.get_feature_names()[1:])

In [106]:
# List of variables to exclude from X
v2exclude= list(
    data_train.columns[
        data_train.columns.str.contains(r'_[23]')]) # Polynomials
v2exclude.extend(
    ['landsf', 'age', 'largestmeetingspace', 'sizesf', # Levels
     'rooms','floors'])
v2exclude.extend(
    ['saleprice', 'log_saleprice', 'id', 'new'])
# Print to make sure no variables are accidentally excluded
print('Variables excluded: ', v2exclude)

Variables excluded:  ['year_2', 'year_3', 'age_2', 'age_3', 'floors_2', 'landsf', 'age', 'largestmeetingspace', 'sizesf', 'rooms', 'floors', 'saleprice', 'log_saleprice', 'id', 'new']


In [107]:
# Non-categorical columns for X
X_train_rest= data_train.drop(cats + v2exclude,
                               axis=1)
X_test_rest= data_test.drop(cats + v2exclude,
                             axis=1)
# Merge categorical and non-categorical variables
X_train= np.concatenate([X_train_rest] + X_train_cats, 
                         axis=1)
X_test= np.concatenate([X_test_rest] + X_test_cats, 
                         axis=1)

# Get feature names
feature_names= list(X_train_rest.columns) \
    + feature_names_cats
# Create target variables
y_train = data_train.loc[:, 'log_saleprice']
y_test = data_test.loc[:, 'log_saleprice']

Let's go ahead and train the model.  Even though I removed the first category of each categorical variable for which I used one-hot-encoding, I also had to remove the intercept in order to get a reasonable prediction on the test set. (With the intercept, I got a negative R^2.)  Even though statisticians have developed tools to dig deeper into collinearity to find out which variables are causing the problem, I don't waste any time on this. As explained above, there is no reason to estimate a model without regularization anyway, except for demonstration purposes.

In [108]:
from sklearn.linear_model import LinearRegression
lr= LinearRegression(fit_intercept=True)
lr.fit(X_train, y_train)
# Get sorted coefficients with feature names
coef= pd.Series(
    lr.coef_,
    index=feature_names) \
    .sort_values(ascending=False)

In [109]:
# Print results for non-categorical variables
pd.DataFrame(coef 
    .loc[~ coef.index.str.startswith('x')]) \
    .sort_index() \
    .style.format('{:.5F}')

,0
allsuites,0.00941
boutique,-0.01180
casino,0.88051
cbd,512181862717.14740
conference,0.25458
convention,0.32425
golf,0.14555
indoorcorridors,0.02813
log1_age,-0.11754
log1_landsf,-0.00131


In [110]:
# R_2 on TRAINING set
y_lr= lr.predict(X_train)
r2_lr_train = r2_score(y_train, y_lr)
r2_lr_train

0.8640254134000869

In [111]:
# R_2 on TEST set
y_lr= lr.predict(X_test)
r2_lr= r2_score(y_test, y_lr)
r2_lr

-8.395883920329746e+19

Let's wait with interpreting these results until the next notebook, where we can compare the results from all models.  Thus, we will store these results for later. 

In [112]:
r2_up = [r2_lr_train, r2_lr]
r2_up

[0.8640254134000869, -8.395883920329746e+19]

# results without intercept

In [113]:
lr2 = LinearRegression(fit_intercept=False)
lr2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [114]:
# R_2 on TRAINING set
y_lr= lr2.predict(X_train)
r2_lr_train = r2_score(y_train, y_lr)
r2_lr_train

0.859678710446465

In [115]:
# R_2 on TEST set
y_lr= lr2.predict(X_test)
r2_lr= r2_score(y_test, y_lr)
r2_lr

0.5791522202341939

In [116]:
X_train.shape

(6949, 1216)

In [117]:
raise Exception ()

Exception: 

#### Logs or squares
Now let's estimate the same type of model but with different predictors: Instead of using all logs for numeric variables, I will follow Das et al (2017) and instead use a square term for age and floors, as well as a the levels (rather than log) for the amount of land included.

In [ ]:
v2include = \
        ['allsuites', 'boutique', 'casino', 'cbd', 
         'conference', 'convention', 'golf', 'indoorcorridors',
         'multiproperty', 'portfolio', 'restaurant', 
         'ski', 'spa', 'landsf', 'age', 'age_2',
         'log1_largestmeetingspace', 'log_sizesf', 'log_rooms', 
         'floors', 'floors_2', 'selfrun']

In [ ]:
# Non-categorical columns for X
X_train_rest= data_train.loc[:, v2include]
X_test_rest= data_test.loc[:, v2include]

# Merge categorical and non-categorical variables
X_train= np.concatenate([X_train_rest] + X_train_cats, 
                         axis=1)
X_test= np.concatenate([X_test_rest] + X_test_cats, 
                         axis=1)

# Get feature names
feature_names= list(X_train_rest.columns) \
    + feature_names_cats
# Create target variables
y_train = data_train.loc[:, 'log_saleprice']
y_test = data_test.loc[:, 'log_saleprice']

In [ ]:
from sklearn.linear_model import LinearRegression
lr= LinearRegression(fit_intercept=False)
lr.fit(X_train, y_train)
# Get sorted coefficients with feature names
coef= pd.Series(
    lr.coef_,
    index=feature_names) \
    .sort_values(ascending=False)

In [ ]:
# Print results for non-categorical variables
pd.DataFrame(coef 
    .loc[~ coef.index.str.startswith('x')]) \
    .sort_index() \
    .style.format('{:.5F}')

In [ ]:
# R_2 on TRAINING set
y_lr= lr.predict(X_train)
r2_lr_train = r2_score(y_train, y_lr)
r2_lr_train

In [ ]:
# R_2 on TEST set
y_lr= lr.predict(X_test)
r2_lr= r2_score(y_test, y_lr)
r2_lr

For comparison, let's print the results from the model using all logs.

In [ ]:
# R_2 for model with all logs 
# (for training and test set, respectively)
r2_up

We see that the two models performed very similarly, especially on the training set. On the test set, the model with polynomial terms performs slightly better than the model with all logs.  However, the difference is likely within the margin of error. (Indeed, when I tried a different specification that used a linear time trend, I got the reverse result.)  Thus, I will stick with the model with all logs for now, since this is more robust to outliers: As already mentioned, polynomials can give predictions that are far off for observations that have variables with extreme values, such as very old buildings.

### Pooled

The next step is to estimate the pooled model without regularization.  To do so, we first repeat the preprocessing steps but do not perform one-hot encoding for the three categorical variables that have a large number of categories.

In [ ]:
# Categorical variables
cats = ['location', 'category', 'operation', 
        'year', 'quarter']
# Categorical variables to exclude
cats_out = ['msa', 'tract', 'saleaffiliation']
# Instantiate one-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore',
                    sparse=False)
# Create lists to store encodings for each variable
X_train_cats = []
X_test_cats = []
feature_names_cats = []
for cat in cats:
    # Encode variable i for TEST data
    ohe_train_i = ohe.fit_transform(
                data_train.loc[:, [cat]])
    # Encode variable i for TRAIN data
    ohe_test_i = ohe.transform(
                data_test.loc[:, [cat]])
    
    # Discard first column and save
    X_train_cats.append(ohe_train_i[:,1:])
    X_test_cats.append(ohe_test_i[:,1:])
    
    # Save feature names
    feature_names_cats.extend(
        ohe.get_feature_names()[1:])

In [ ]:
# List of variables to exclude from X
v2exclude= list(
    data_train.columns[
        data_train.columns.str.contains(r'_[23]')]) # Polynomials
v2exclude.extend(
    ['landsf', 'age', 'largestmeetingspace', 'sizesf', # Levels
     'rooms','floors'])
v2exclude.extend(
    ['saleprice', 'log_saleprice', 'id', 'new'])
# Print to make sure no variables are accidentally excluded
print('Variables excluded: ', v2exclude)

In [ ]:
# Non-categorical columns for X
X_train_rest= data_train.drop(cats + cats_out + v2exclude,
                               axis=1)
X_test_rest= data_test.drop(cats + cats_out + v2exclude,
                             axis=1)
# Merge categorical and non-categorical variables
X_train= np.concatenate([X_train_rest] + X_train_cats, 
                         axis=1)
X_test= np.concatenate([X_test_rest] + X_test_cats, 
                         axis=1)

# Get feature names
feature_names= list(X_train_rest.columns) \
    + feature_names_cats
# Create target variables
y_train = data_train.loc[:, 'log_saleprice']
y_test = data_test.loc[:, 'log_saleprice']

Now we are ready again to estimate the model.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(fit_intercept=True)
lr.fit(X_train, y_train)
# Get sorted coefficients with feature names
coef= pd.Series(
    lr.coef_,
    index=feature_names) \
    .sort_values(ascending=False)
coef

Let's print the explained variance for training and test set.

In [ ]:
# R_2 on TRAINING set
y_lr= lr.predict(X_train)
r2_lr_train = r2_score(y_train, y_lr)
r2_lr_train

In [ ]:
# R_2 on TEST set
y_lr= lr.predict(X_test)
r2_lr= r2_score(y_test, y_lr)
r2_lr

In [ ]:
# Save performance
r2_p = [r2_lr_train, r2_lr]
r2_p

Surprisingly, the model does a little better on the test set. On average, we would expect it to do worse. However, there is variance to our estimated performance on training and test set.  In our case, this variance is increased by the fact that we don't have a huge sample size (the test set contains about 1300 observations), and that we are using a single validation set rather than cross-validation in order to take into account the time-series nature of the data. Even more importantly, the training and test set do not come from the same distribution, because we split the data based on time.  It seems that the newer data were easier to predict, maybe because we have more data for the intercept of year.

## Linear model with elastic net regularization
### Unpooled

Let's perform the preprocessing steps again.  By contrast to the models without regularization, we do not get rid of the first category for each categorical variables.  The reason this is not necessary is that regularization is an alternative way to avoid collinearity.

In [ ]:
# Categorical variables
cats = ['msa', 'tract', 'location', 'category',
        'saleaffiliation', 'operation', 'year',
        'quarter']
# Perform one-hot encoding
ohe = OneHotEncoder(handle_unknown='ignore',
                    sparse=False)
X_train_cats = ohe.fit_transform(
    data_train.loc[:, cats])        
X_test_cats = ohe.transform(
    data_test.loc[:, cats])

In [ ]:
# List of variables to exclude from X
v2exclude= list(
    data_train.columns[
    data_train.columns.str.contains(r'_[23]')]) # Polynomials
v2exclude.extend(
    ['landsf', 'age', 'largestmeetingspace', 'sizesf', # Levels
     'rooms','floors'])
v2exclude.extend(
    ['saleprice', 'log_saleprice', 'id', 'new'])
# Print to make sure no variables are accidentally excluded
print('Variables excluded: ', v2exclude)

In [ ]:
# Non-categorical columns for X
X_train_rest= data_train.drop(cats + v2exclude,
                               axis=1)
X_test_rest= data_test.drop(cats + v2exclude,
                             axis=1)
# Merge categorical and non-categorical variables
X_train_up = np.concatenate([X_train_rest, X_train_cats], 
                         axis=1)
X_test_up = np.concatenate([X_test_rest, X_test_cats], 
                         axis=1)
# X_train_up = X_train_up.drop(
#     v2exclude, axis=1)
# X_test_up = X_test_up.drop(
#     v2exclude, axis=1)

# Get feature names
feature_names= list(X_train_rest.columns) \
    + list(ohe.get_feature_names())

# Create target variables
y_train = data_train.loc[:, 'log_saleprice']
y_test = data_test.loc[:, 'log_saleprice']

We can no go ahead and train the model.  Like for gradient boosting, will use Hyperopt to perform Bayesian hyperparameters optimization.  It probably does not lead to any big increase in performance or decrease in computational cost compared to a brute-force search, because computational costs for a linear model are relatively low anyways for a data set of our size. But since it is not too much of a burden to adapt the  code from XGBoost, we might as well reuse it just in case.

In [ ]:
# Function to carry out hyperparameter optimization
def find_best_hp(LEARNER, space, model_name, 
                 X_train, y_train, 
                 adjust_params=None,n_folds=5, n_jobs=-1, max_evals=20):
    """Find best hyperparameters for a given regressor and search space."""
    
    # Trials object to track progress (not currently used)
    trials = Trials()

    # CSV file to track progress
    progress_file_path = '../progress_' + model_name + '.csv'
    with open(progress_file_path, 'w') as file:
        writer = csv.writer(file)
        # Write header to the file
        writer.writerow(['loss', 'params'])

    # Objective function to minimize
    def objective(params, LEARNER=LEARNER, 
                  progress_file_path=progress_file_path,
                  n_folds=n_folds, n_jobs=n_jobs):
        """Objective function to minimize"""
        
        # Adjust parameters, if specified
        if adjust_params is not None:
            params = adjust_params(params)
    
        # Instantiate LEARNER
        learner = LEARNER(**params)
        
        ## Generate indices for cross-validation
        # If only one "fold" is desired, split into train and validation set
        if n_folds == 1: 
            cv = ShuffleSplit(n_splits=1, test_size=.2, 
                                        random_state=1)
        # Otherwise, generate indices for proper cross-validation split
        else:  
            cv = KFold(n_folds, random_state=1)

        # Compute average precision through CV / validation set
        score = cross_val_score(learner, X_train, y_train, cv=cv,
                                scoring='r2', n_jobs=n_jobs)
        # Compute loss as the negative mean of the average precision scores
        # (since hyperopt can only minimize a function)
        loss = -score.mean()
        
        # Save results to csv file
        with open(progress_file_path, 'a') as file:
            writer = csv.writer(file)
            writer.writerow([loss, params])
        
        # Return results
        return {'loss': loss, 'params': params, 'status': STATUS_OK}
    
    # Minimize objective
    best = fmin(objective, space, algo=tpe.suggest,
                max_evals=max_evals, trials=trials)

    # Get the values of the optimal parameters
    best_params = space_eval(space, best)
    # Adjust best parameters, if specified
    if adjust_params is not None:
        best_params = adjust_params(best_params)

    # Re-fit the model with the optimal hyperparamters
    learner = LEARNER(**best_params)
    learner.fit(X_train, y_train)
    
    # Save model to disk
    joblib.dump(learner, '../saved_models/' + model_name + '.joblib')
    
    # Print best parameters
    print(best_params)

In [ ]:
MAX_EVALS = 50
N_FOLDS = 20

# Search space (also includes constant parameters)
space = {
    'max_iter': 1000,
    'tol':1E-3,
    'random_state': 1,
    'alpha': hp.lognormal('alpha', np.log(1E-4), 4),
    'l1_ratio': hp.uniform('l1_ratio', 0, 1)
}

# Find best hyperparameters (defined above)
find_best_hp(ElasticNet, space, model_name='el_up',
              X_train=X_train_up, y_train=y_train,
              max_evals=MAX_EVALS, n_jobs=3, n_folds=N_FOLDS)

In [ ]:
# Load saved model, if necessary
el_up = joblib.load('../saved_models/el_up.joblib')

In [ ]:
# Load full results from progress file
el_up_results = pd.read_csv('../progress_el_up.csv')

# Get validation score
# Extract AP for each iteration
r2_el_up = - el_up_results.loss

# Plot r2 per iteration
r2_el_up.plot()
plt.title('Performance on test set')
plt.ylabel('R_2')
plt.xlabel('Iteration');

This reveals that optimizing hyperparameters to regularize a linear model is relatively simple, so performance seems pretty stable without much improvement for further iterations.

In [ ]:
# R_2 on TRAINING set
y_el_up = el_up.predict(X_train_up)
r2_el_up_train = r2_score(y_train, y_el_up)
r2_el_up_train

In [ ]:
# R_2 on TEST set
y_el_up = el_up.predict(X_test_up)
r2_el_up = r2_score(y_test, y_el_up)
r2_el_up

In [ ]:
# Save performance
r2_up_r = [r2_el_up_train, r2_el_up]
r2_up_r

Examining the explained variance reveals that regularization gave us a boost in performance.  However, as mentioned above, we will save a comparison for a final notebook.

Let's print the coefficients, ignoring the intercepts for all categorical variables in order to limit the size of the table.

In [ ]:
# Get sorted coefficients with feature names
coef= pd.Series(
    el_up.coef_,
    index=feature_names) \
    .sort_values(ascending=False)

# Print results for non-categorical variables
pd.DataFrame(coef 
    .loc[~ coef.index.str.startswith('x')]) \
    .sort_index() \
    .style.format('{:.5F}')

### Pooled
Finally, we will re-estimate the model for the pooled case. Again, we do not drop the first category when performing one-hot encoding, since regularization effectively deals with collinearity.

In [ ]:
# Categorical variables to include
cats = ['location', 'category', 'operation', 
        'year', 'quarter']
# Categorical variables to exclude
cats_out = ['msa', 'tract', 'saleaffiliation']
# Perform one-hot encoding
ohe = OneHotEncoder(handle_unknown='ignore',
                    sparse=False)
X_train_cats = ohe.fit_transform(
    data_train.loc[:, cats])        
X_test_cats = ohe.transform(
    data_test.loc[:, cats])

In [ ]:
# List of variables to exclude from X
v2exclude= list(
    data_train.columns[
    data_train.columns.str.contains(r'_[23]')]) # Polynomials
v2exclude.extend(
    ['landsf', 'age', 'largestmeetingspace', 'sizesf', # Levels
     'rooms','floors'])
v2exclude.extend(
    ['saleprice', 'log_saleprice', 'id', 'new'])
# Print to make sure no variables are accidentally excluded
print('Variables excluded: ', v2exclude)

In [ ]:
# Non-categorical columns for X
X_train_rest= data_train.drop(cats  + cats_out+ v2exclude,
                               axis=1)
X_test_rest= data_test.drop(cats + cats_out+ v2exclude,
                             axis=1)
# Merge categorical and non-categorical variables
X_train_p = np.concatenate([X_train_rest, X_train_cats], 
                         axis=1)
X_test_p = np.concatenate([X_test_rest, X_test_cats], 
                         axis=1)

In [ ]:
# Get feature names
feature_names= list(X_train_rest.columns) \
    + list(ohe.get_feature_names())

# Create target variables
y_train = data_train.loc[:, 'log_saleprice']
y_test = data_test.loc[:, 'log_saleprice']

In [ ]:
MAX_EVALS = 50
N_FOLDS = 20

# Search space (also includes constant parameters)
space = {
    'max_iter': 1000,
    'tol':1E-3,
    'random_state': 1,
    'alpha': hp.lognormal('alpha', np.log(1E-4), 4),
    'l1_ratio': hp.uniform('l1_ratio', 0, 1)
}

# Find best hyperparameters (defined above)
find_best_hp(ElasticNet, space, model_name='el_p',
              X_train=X_train_p, y_train=y_train,
              max_evals=MAX_EVALS, n_jobs=3, n_folds=N_FOLDS)

In [ ]:
# Load saved model, if necessary
el_p = joblib.load('../saved_models/el_p.joblib')

In [ ]:
# Load full results from progress file
el_results = pd.read_csv('../progress_el_p.csv')

# Get validation score
# Extract AP for each iteration
r2_el_p = - el_results.loss

# Plot r2 per iteration
r2_el_p.plot()
plt.title('Performance on test set')
plt.ylabel('r2')
plt.xlabel('Iteration');

When examining how the performance change over the iterations of the hyperparameters optimization, we see that it stayed almost stable.  This is due to the fact that hyperparameters optimization for this case is easy (compared, e.g., what we saw for gradient boosting, where we optimized over a higher-dimensional space.)

In [ ]:
# R_2 on TRAINING set
y_el_p = el_p.predict(X_train_p)
r2_el_p_train = r2_score(y_train, y_el_p)
r2_el_p_train

In [ ]:
# R_2 on TEST set
y_el_p = el_p.predict(X_test_p)
r2_el_p = r2_score(y_test, y_el_p)
r2_el_p

In [ ]:
# Save performance
r2_p_r = [r2_el_p_train, r2_el_p]
r2_p_r

In [ ]:
# Get sorted coefficients with feature names
coef= pd.Series(
    el_p.coef_,
    index=feature_names) \
    .sort_values(ascending=False)

# Print results for non-categorical variables
pd.DataFrame(coef 
    .loc[~ coef.index.str.startswith('x')]) \
    .sort_index() \
    .style.format('{:.5F}')

## Save performance
Finally, we concatenate the performance of all linear models into a data frame and save it to disk. We will load this in the final notebook, where we compare the performance of these models with each other and with the performance of gradient boosting.

In [ ]:
# Combine all R_2 scores from OLS
r2_lin = pd.DataFrame([r2_up, r2_p, r2_up_r, r2_p_r],
             columns=['Train', 'Test'],
             index=['OLS, unpooled, no reg.',
                    'OLS, pooled, no reg.',
                    'linear regression, unpooled, elastic net reg.',
                    'linear regression, pooled, elastic net reg.']
)
joblib.dump(r2_lin, '../saved_models/r2_lin.joblib')

In [ ]:
import os
os.system('jupyter nbconvert --to html 2_modeling_linear.ipynb')

In [ ]:
os.getcwd()